![image](https://www.ewatercycle.org/assets/logo.png)

# Case study 0: Hello worlds. Run two models on the same forcing data and compare output (discharge)
This notebooks demonstrates how to use models in eWaterCycle by showing how to run two different models for the same region (Merrimack) and on the same forcing data (ERA5: 1990).

The models used in this notebook are:

- WFlow (details)
- LISFLood (details)

## Import statements
We'll be using the following modules

In [46]:
import logging
logger = logging.getLogger('grpc4bmi.bmi_grpc_client')
logger.setLevel(logging.WARNING)

In [47]:
from cftime import num2date
import pandas as pd
import xarray as xr
from pathlib import Path

from ewatercycle import CFG
from ewatercycle.models import Wflow
from ewatercycle.models import Lisflood
from ewatercycle.analysis import hydrograph
from ewatercycle.observation.grdc import get_grdc_data
from ewatercycle.models.lisflood import LisfloodParameterSet

In [48]:
CFG.load_from_file('./ewatercycle.yaml')

## Wflow

In [37]:
# create wflow instance
model = Wflow()

cfg_file, cfg_dir = model.setup(
    cfg_dir='/projects/0/wtrcycle/comparison/wflow_parameterset/calibrated_parameterset/merrimack/',
    cfg_file='./wflow_sbm_Merrimack_ERA5_warmup.ini',
    API={"RiverRunoff": "2, m^3/s"}
)
print(cfg_dir)

Working directory created: /scratch/shared/ewatercycle/wflow_20210421_173909
Created /scratch/shared/ewatercycle/wflow_20210421_173909/wflow_ewatercycle.ini.
Running ewatercycle-wflow-grpc4bmi.sif singularity container on port 42344
Started wflow container with working directory /scratch/shared/ewatercycle/wflow_20210421_173909
/scratch/shared/ewatercycle/wflow_20210421_173909


In [40]:
# initialize
model.initialize(str(cfg_file))

# get metadata
print(model.output_var_names)

('RiverRunoff',)


In [42]:
discharge = []
while model.time < model.end_time:
    model.update()
    discharge.append(model.get_value_as_xarray('RiverRunoff'))
    
wflow_output = xr.concat(discharge, dim='time')

In [44]:
wflow_output

<xarray.DataArray 'RiverRunoff' (time: 365, latitude: 242, longitude: 154)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])
Coordinates:
  * longitude  (longitude) float64 -72.14 -72.13 -72.12 ... -70.88 -70.87 -70.86
  * latitude   (latitude) float64 42.2 42.21 42.22 42.23 ... 44.2 44.2 44.21
  * time       (time) object 1990-01-01 12:00:00 ... 1990-12-31 12:00:00
Attributes:
    units:     m^3/s

In [45]:
# stop the model
model.finalize()
del(model)

In [ ]:
def guess_outlet_gridpoint(data, lat, lon, pad=0.2):
    """Find the gridpoint nearby (lat, lon) that's most likely a river."""
    box = data.sel(
        longitude=slice(lon - pad, lon + pad),
        latitude=slice(lat - pad, lat + pad),
    )
    boxmax = box.max(["longitude", "latitude"])
    return boxmax

## Lisflood

In [49]:
parameterset = LisfloodParameterSet(
    root=Path('/projects/0/wtrcycle/comparison/lisflood_input/Lisflood01degree_masked'),
    mask=Path('/projects/0/wtrcycle/comparison/recipes_auxiliary_datasets/LISFLOOD/model_mask.nc'),
    config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisflood.xml'),
    lisvap_config_template=Path('/projects/0/wtrcycle/comparison/lisflood_input/settings_templates/settings_lisvap.xml'),
)

In [50]:
# create lisflood instance
model = Lisflood()

# setup model
# lisflood_ERA5_*_1990_1990.nc data can be copied 
# from /projects/0/wtrcycle/comparison/forcing/lisflood
forcing_dir = Path('lisflood_forcing_data')
config_file, config_dir = model.setup(forcing_dir, parameterset)
print(config_dir)

# run lisvap
# lisvap_results = model.run_lisvap(forcing_dir)

Running ewatercycle-lisflood-grpc4bmi.sif singularity container on port 42402
/scratch/shared/ewatercycle/lisflood_20210421_180752


In [51]:
# initialize
model.initialize(str(config_file))

# get metadata
print(model.output_var_names)

('Discharge',)


In [52]:
# run the model
discharge = []
while model.time < model.end_time:
    model.update()
    discharge.append(model.get_value_as_xarray('Discharge'))

lisflood_output = xr.concat(discharge, dim='time')

In [53]:
lisflood_output

<xarray.DataArray 'Discharge' (time: 363, latitude: 1500, longitude: 3600)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]])
Coordinates:
  * longitude  (longitude) float64 -179.9 -179.8 -179.8 ... 179.7 179.8 179.9
  * latitude   (latitude) float64 89.95 89.85 89.75 ... -59.75 -59.85 -59.95
  * time       (time) object 1990-01-03 00:00:00 ... 1990-12-31 00:00:00
Attributes:
    units:    m^3/s

In [56]:
# Get time information
start = num2date(model.start_time, model.time_units).strftime("%Y-%m-%d")
end = num2date(model.end_time, model.time_units).strftime("%Y-%m-%d")

# stop the model
del model.bmi

In [63]:
# select outlet pixel
discharge = lisflood_output.isel(dict(longitude=1086, latitude=473))
discharge.to_dataframe().rename(columns={"Discharge": "lisflood"})

,longitude,latitude,lisflood
time,,,
1990-01-03 00:00:00,-71.35,42.65,9.061750
1990-01-04 00:00:00,-71.35,42.65,15.274644
1990-01-05 00:00:00,-71.35,42.65,30.082788
1990-01-06 00:00:00,-71.35,42.65,51.922786
1990-01-07 00:00:00,-71.35,42.65,84.816580
...,...,...,...
1990-12-27 00:00:00,-71.35,42.65,318.036922
1990-12-28 00:00:00,-71.35,42.65,398.285179
1990-12-29 00:00:00,-71.35,42.65,491.402610


## Observation

In [ ]:
# Setting GRDC_DATA_HOME so that `get_grdc_data` can find them:
os.environ["GRDC_DATA_HOME"] = str(
    PROJECT_HOME / "GRDC" / "GRDC_GCOSGTN-H_27_03_2019"
)

# Get grdc observations
ds = get_grdc_data("4147380", start, end)

# Convert grdc observations to DataFrame
observations = ds.to_dataframe().rename(columns={"streamflow": "GRDC"})
observations.index = observations.index.date
observations.index.name = "time"
observations

## Hydrograph

In [ ]:
pd.concat([wflow_discharge, lisflood_discharge, observations], axis=1, sort=False)
hydrograph(df, reference='GRDC')